In [ ]:
%%time
import pandas as pd
import seaborn as sns
import os
import hashlib
import math
import csv
import itertools
import pandas_bokeh
from collections import Counter
import matplotlib.pyplot as plt

pandas_bokeh.output_notebook()
import jupyter_black

jupyter_black.load()

<h1>Ghidra 11.4.2 and PyGhidra</h1>

In [ ]:
!wget https://github.com/therealsaumil/emux/raw/master/files/emux/TRI227WF/rootfs.tar.bz2
!wget https://github.com/therealsaumil/emux/raw/master/files/emux/AC15/squashfs-root.tar.bz2

In [ ]:
!bzip2 -d ./rootfs.tar.bz2
!bzip2 -d ./squashfs-root.tar.bz2
!tar -xvf ./rootfs.tar
!tar -xvf ./squashfs-root.tar

In [ ]:
!mkdir ./AC15
!cp ./squashfs-root/bin/httpd ./AC15/AC15_httpd
!cp ./rootfs/usr/bin/webs ./AC15/TRI227WF_webs

In [ ]:
import pyghidra
import os
import csv
from operator import itemgetter

pyghidra.start()

import ghidra
from ghidra.app.util.headless import HeadlessAnalyzer
from ghidra.program.flatapi import FlatProgramAPI
from ghidra.base.project import GhidraProject
from java.lang import String
from ghidra.program.util import DefinedDataIterator, CyclomaticComplexity
from ghidra.program.model.listing import Function
from ghidra.program.model.symbol import SourceType
from ghidra.util.exception import CancelledException

# Define dangerous functions
dangerous_functions = ["system", "execve", "execle", "execvp", "execlp", "doSystemCmd"]


def format_high_complexity_funcs(funcs):
    """Format the top 10 high complexity functions as a string."""
    return "; ".join([f"{name}({cc})" for name, cc in funcs])


def analyze_binary(binary_path):
    try:
        with pyghidra.open_program(binary_path) as flat_api:
            # Get program and listing
            current_program = flat_api.getCurrentProgram()
            listing = current_program.getListing()

            # Get basic program info
            files = current_program.getName()
            arches = current_program.getLanguage().toString()
            sha256 = current_program.getExecutableSHA256()
            md5 = current_program.getExecutableMD5()
            total_insn = listing.getNumInstructions()

            # Get functions and calculate metrics
            all_funcs = list(listing.getFunctions(True))
            total_cc = 0
            system_xrefs_details = []
            monitor = flat_api.getMonitor()

            # Analyze dangerous functions and their xrefs
            ref_manager = current_program.getReferenceManager()
            for func in all_funcs:
                if func.getName() in dangerous_functions:
                    entry_point = func.getEntryPoint()
                    references = ref_manager.getReferencesTo(entry_point)
                    for xref in references:
                        ref_func = listing.getFunctionContaining(xref.getFromAddress())
                        if ref_func:
                            detail = f"{xref.getFromAddress()} ({ref_func.getName()})"
                            system_xrefs_details.append(detail)

            num_calls_in_system_xrefs = len(system_xrefs_details)

            # Calculate cyclomatic complexity metrics
            cc_calculator = CyclomaticComplexity()
            complexity_funcs = []
            for func in all_funcs:
                try:
                    cc = cc_calculator.calculateCyclomaticComplexity(func, monitor)
                    total_cc += cc
                    # Store all functions with their complexity
                    complexity_funcs.append((func.getName(), cc))
                except CancelledException:
                    print(
                        f"Warning: Complexity calculation cancelled for function {func.getName()}"
                    )

            num_funcs = len(all_funcs)
            average_cc = total_cc / num_funcs if num_funcs > 0 else 0

            # Sort functions by complexity and get top 10
            top_complex_funcs = sorted(
                complexity_funcs, key=itemgetter(1), reverse=True
            )[:10]

            # Save results to CSV
            csv_file_path = "./ghidratest.csv"
            with open(csv_file_path, mode="a", newline="") as csv_file:
                fieldnames = [
                    "File",
                    "Architecture",
                    "SHA256",
                    "MD5",
                    "Total_Instructions",
                    "Total_Functions",
                    "System_Xrefs",
                    "Total_System_Xrefs",
                    "Average_Cyclomatic_Complexity",
                    "Top_10_Complex_Functions",  # New field
                ]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                if csv_file.tell() == 0:
                    writer.writeheader()

                writer.writerow(
                    {
                        "File": files,
                        "Architecture": arches,
                        "SHA256": sha256,
                        "MD5": md5,
                        "Total_Instructions": total_insn,
                        "Total_Functions": num_funcs,
                        "System_Xrefs": "; ".join(system_xrefs_details),
                        "Total_System_Xrefs": num_calls_in_system_xrefs,
                        "Average_Cyclomatic_Complexity": round(average_cc, 2),
                        "Top_10_Complex_Functions": format_high_complexity_funcs(
                            top_complex_funcs
                        ),
                    }
                )

    except Exception as e:
        print(f"Error loading binary {binary_path}: {str(e)}. Skipping file.")
    except Exception as e:
        print(f"Error analyzing binary: {str(e)}")
        raise


def scan_directory(directory_path):
    # Scan the directory for binaries and analyze each one
    for root, _, files in os.walk(directory_path):
        for file in files:
            binary_path = os.path.join(root, file)
            if os.path.isfile(binary_path):  # Make sure it's a file
                print(f"Analyzing binary: {binary_path}")
                analyze_binary(binary_path)


if __name__ == "__main__":
    # Change this path to the directory you want to scan
    directory_path = "./AC15/"
    scan_directory(directory_path)

In [ ]:
%%time
df = pd.read_csv("./ghidratest.csv", header=None)

<h2>Naming Pandas Columns</h2>

In [ ]:
df

In [ ]:
%%time
df.columns = [
    "File",
    "Architecture",
    "SHA256",
    "MD5",
    "Strings",
    "Functions",
    "System_Xrefs",
    "Total_System_Xrefs",
    "Average_Cyclomatic_Complexity",
    "Top_10_Complex_Functions",
]

In [ ]:
df = pd.read_csv(
    "ghidratest.csv",
    dtype={
        "Total_Instructions": int,
        "Total_Functions": int,
        "Total_System_Xrefs": int,
        "Average_Cyclomatic_Complexity": float,
    },
)

In [ ]:
df.fillna("None", inplace=True)

<h2>Verify the Pandas Output</h2>

In [ ]:
%%time
df

<h2>Checking Pandas Datatypes</h2>

In [ ]:
%%time
df.dtypes

<h2>Changing Datatypes to String</h2>

In [ ]:
%%time
df["Total_Instructions"] = df["Total_Instructions"].astype(str)
df["Total_Functions"] = df["Total_Functions"].astype(str)
df["Architecture"] = df["Architecture"].astype(str)
df["File"] = df["File"].astype(str)
df["SHA256"] = df["SHA256"].astype(str)
df["MD5"] = df["MD5"].astype(str)
df["Architecture"] = df["Architecture"].astype(str)
df["System_Xrefs"] = df["System_Xrefs"].astype(str)

<h2>Searching for Features</h2>

In [ ]:
df[df["Architecture"].str.contains("little", na=False)]

<h2>Using Query to Search for Features</h2>

In [ ]:
df.query("Average_Cyclomatic_Complexity > 3")

In [ ]:
df_sorted = df.sort_values(by="Total_System_Xrefs", ascending=False)

In [ ]:
%%time
df_sorted.plot_bokeh.bar(
    x="File",
    y="Total_System_Xrefs",
    figsize=(900, 700),
    title="Potentially Dangerous Calls To System",
    xlabel="Binary",
    ylabel="Total",
    color="#2196f3",
    vertical_xlabel=True,
)

<h2>Creating Charts Using Pandas Bokeh</h2>

In [ ]:
df.plot_bokeh.bar(
    x="File",
    y=["Average_Cyclomatic_Complexity"],
    figsize=(900, 700),
    title="Average Cyclomatic Complexity",
    xlabel="File",
    ylabel="Average Cyclomatic Complexity",
    color="#2196f3",
    vertical_xlabel=True,
)

In [ ]:
# Process the Top_10_Complex_Functions column
def extract_func_data(func_str):
    # Split the string into individual function entries
    funcs = func_str.split("; ")

    # Extract function names and complexity scores
    names = []
    scores = []
    for func in funcs:
        if func:  # Check if the function entry is not empty
            name, score = func.strip("()").split("(")
            names.append(name)
            scores.append(float(score))

    return pd.DataFrame({"Function_Name": names, "Complexity": scores})


# Create a visualization for each binary
for idx, row in df.iterrows():
    binary_name = row["File"]
    func_data = extract_func_data(row["Top_10_Complex_Functions"])

    # Create bar plot
    plot = func_data.plot_bokeh(
        kind="bar",
        x="Function_Name",
        y="Complexity",
        title=f"Top 10 High Complexity Functions in {binary_name}",
        xlabel="Function Name",
        ylabel="Cyclomatic Complexity",
        figsize=(1000, 600),
        color="#2196f3",
        vertical_xlabel=True,
        show_figure=True,
    )

<h2>Reference Material</h2>

- 10 Minutes to Pandas: https://pandas.pydata.org/docs/user_guide/10min.html
- Pandas Cookbook: https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook
- Ghidra API: https://ghidra.re/ghidra_docs/api/index.html
- PyGhidra: https://github.com/NationalSecurityAgency/ghidra/tree/master/Ghidra/Features/PyGhidra
- EMUX: https://github.com/therealsaumil/emux
- Ghidra Snippets: https://github.com/HackOvert/GhidraSnippets
- Auditing system calls for command injection vulnerabilities using Ghidra's PCode: https://youtu.be/UVNeg7Vqytc
- cetfor/SystemCallAuditorGhidra.py: https://github.com/HackOvert/PotentiallyVulnerable/blob/main/CWE-78/SystemCallAuditorGhidra.py